# Dictionaries example
For a more in-depth explanation read
https://www.gurobi.com/documentation/9.0/quickstart_windows/py_python_dictionary_examp.html

In [6]:
import gurobipy as gp
from gurobipy import GRB

## Custom gurobipy classes

The `multidict` class allows to initialize multiple dictionaries at once.
It takes as an input a dictionary in which each entry's value is a list of length n. The multidict class will then split these lists, creating n seperate dictionaries that share the same keys.

In [7]:
names, lower, upper = gp.multidict(dict(x=[0,1], y=[1,2], z=[0,3]))
print(names)
print(lower)
print(upper)

['x', 'y', 'z']
{'x': 0, 'y': 1, 'z': 0}
{'x': 1, 'y': 2, 'z': 3}


The `tuplelist` class stores a list of tuples and is a subclass of the standard list class. The .select function is an efficient way to extract specific tuples based on the given criteria. It works with wildcards ('*') and multi-criteria inputs (lists)

In [16]:
l = gp.tuplelist([(1, 2), (1, 3), (2, 3), (2, 4)])
l.select(1, '*')  # will give all entries that have 1 in the first position

<gurobi.tuplelist (2 tuples, 2 values each):
 ( 1 , 2 )
 ( 1 , 3 )
>

In [17]:
l.select('*', [2, 4])  # will return all entries that have 2 or 4 at the second position

<gurobi.tuplelist (2 tuples, 2 values each):
 ( 1 , 2 )
 ( 2 , 4 )
>

The `tupledict` class is a subclass of the python dictionaries that uses `tuplelist` as keys (so the .select method can be used to select subsets of the `tupledict`). In a tupledict, Gurobi decision variables can be stored that correspond to the tuple key. In a VRP, e.g., the key tuple may be (node i, node j, vehicle v) and the tupledict class allows to easily select only those arcs that originate from node i. 

In [26]:
l = list([(1, 2), (1, 3), (2, 3), (2, 4)])
m = gp.Model(name='dict1')
d = m.addVars(l, name='d')  # the addVars method creates Variables and returns them as a tupledict
m.update()
type(d)

gurobipy.tupledict

In [27]:
d

{(1, 2): <gurobi.Var d[1,2]>,
 (1, 3): <gurobi.Var d[1,3]>,
 (2, 3): <gurobi.Var d[2,3]>,
 (2, 4): <gurobi.Var d[2,4]>}

In [31]:
sum(d.select(1, '*'))  # the tupledict can be used to create linear expressions of the variables

<gurobi.LinExpr: d[1,2] + d[1,3]>

In [32]:
d.sum(1, '*')  # is the faster way of doing the same

<gurobi.LinExpr: d[1,2] + d[1,3]>

In [33]:
coeff = {(1,2):5, (2,3):7}
# coeff = {l[0]: 5, l[2]:7}
d.prod(coeff)   # the .prod method of the tupledict class handles expressions where variables are weighted differently.
                # the input dictionary has the same tuple keys as the variables themselves

<gurobi.LinExpr: 5.0 d[1,2] + 7.0 d[2,3]>

In [34]:
d.prod(coeff, 2)  # this method also accepts a filter

<gurobi.LinExpr: 7.0 d[2,3]>